In [ ]:
import os
import cv2
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QPushButton, QVBoxLayout, QHBoxLayout, \
    QFileDialog, QMessageBox, QSlider, QMainWindow, QScrollArea
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt

class ImageCompressionApp(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Image Compression App")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.layout = QVBoxLayout()
        self.central_widget.setLayout(self.layout)

        self.label_info = QLabel("Select images to compress:")
        self.layout.addWidget(self.label_info)

        self.scroll_area = QScrollArea()
        self.scroll_area.setWidgetResizable(True)
        self.scroll_area.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
        self.scroll_area.setVerticalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        self.layout.addWidget(self.scroll_area)

        self.scroll_widget = QWidget()
        self.scroll_layout = QVBoxLayout()
        self.scroll_widget.setLayout(self.scroll_layout)
        self.scroll_area.setWidget(self.scroll_widget)

        self.btn_add_images = QPushButton("Add Images")
        self.btn_add_images.clicked.connect(self.add_images)
        self.layout.addWidget(self.btn_add_images)

        self.quality_slider_label = QLabel("Compression Quality:")
        self.layout.addWidget(self.quality_slider_label)

        self.quality_slider = QSlider(Qt.Horizontal)
        self.quality_slider.setRange(10, 100)  # Avoid 0% quality
        self.quality_slider.setValue(80)  # Default compression quality
        self.quality_slider.setTickInterval(10)
        self.quality_slider.setTickPosition(QSlider.TicksBelow)
        self.layout.addWidget(self.quality_slider)

        self.btn_compress_all = QPushButton("Compress All Images")
        self.btn_compress_all.clicked.connect(self.compress_all_images)
        self.layout.addWidget(self.btn_compress_all)

        self.status_label = QLabel("")
        self.layout.addWidget(self.status_label)

        self.image_widgets = []
        self.input_image_paths = []

    def add_images(self):
        options = QFileDialog.Options()
        file_names, _ = QFileDialog.getOpenFileNames(self, "Select Images", "",
                                                     "Image files (*.jpg *.jpeg *.png)", options=options)
        if file_names:
            for file_name in file_names:
                self.input_image_paths.append(file_name)
                self.add_image_widget(file_name)

    def add_image_widget(self, file_name):
        image_widget = QWidget()
        image_layout = QHBoxLayout()
        image_widget.setLayout(image_layout)

        image_label = QLabel()
        pixmap = QPixmap(file_name)
        pixmap = pixmap.scaledToWidth(200)  # Resize preview image
        image_label.setPixmap(pixmap)
        image_label.setAlignment(Qt.AlignCenter)
        image_layout.addWidget(image_label)

        remove_btn = QPushButton("Remove")
        remove_btn.clicked.connect(lambda _, widget=image_widget: self.remove_image_widget(widget))
        image_layout.addWidget(remove_btn)

        self.image_widgets.append(image_widget)
        self.scroll_layout.addWidget(image_widget)

    def remove_image_widget(self, widget):
        self.image_widgets.remove(widget)
        self.scroll_layout.removeWidget(widget)
        widget.deleteLater()

    def bytes_to_kb_mb(self, size_in_bytes):
        kb = size_in_bytes / 1024
        mb = kb / 1024
        return kb, mb

    def compress_all_images(self):
        if not self.input_image_paths:
            QMessageBox.warning(self, "Error", "Please add images to compress.")
            return

        quality = self.quality_slider.value()

        if not (10 <= quality <= 100):
            QMessageBox.warning(self, "Error", "Quality slider value must be between 10 and 100.")
            return

        for input_image_path in self.input_image_paths:
            if not os.path.exists(input_image_path):
                QMessageBox.critical(self, "Error", f"File does not exist: {input_image_path}")
                continue

            image = cv2.imread(input_image_path)
            if image is None:
                QMessageBox.critical(self, "Error", f"Unable to read image from {input_image_path}. Check file format or path.")
                continue

            output_directory = os.path.dirname(input_image_path)
            output_file_name = os.path.splitext(os.path.basename(input_image_path))[0] + "_compressed.jpg"
            output_image_path = os.path.join(output_directory, output_file_name)

            jpeg_params = [cv2.IMWRITE_JPEG_QUALITY, quality]
            success = cv2.imwrite(output_image_path, image, jpeg_params)
            if not success:
                QMessageBox.critical(self, "Error", f"Failed to save compressed image to {output_image_path}")
                continue

            # Calculate and display file sizes
            original_size = os.path.getsize(input_image_path)
            compressed_size = os.path.getsize(output_image_path)

            original_kb, original_mb = self.bytes_to_kb_mb(original_size)
            compressed_kb, compressed_mb = self.bytes_to_kb_mb(compressed_size)
            size_reduction = original_size - compressed_size
            reduction_percentage = (size_reduction / original_size) * 100 if original_size > 0 else 0

            # Print debug information
            print(f"Original Size: {original_size} bytes ({original_kb:.2f} KB, {original_mb:.2f} MB)")
            print(f"Compressed Size: {compressed_size} bytes ({compressed_kb:.2f} KB, {compressed_mb:.2f} MB)")
            print(f"Size Reduction: {size_reduction} bytes ({reduction_percentage:.2f}%)")

            self.status_label.setText(f"Compressed image saved to {output_image_path}\n"
                                      f"Original size: {original_kb:.2f} KB ({original_mb:.2f} MB)\n"
                                      f"Compressed size: {compressed_kb:.2f} KB ({compressed_mb:.2f} MB)\n"
                                      f"Size reduction: {reduction_percentage:.2f}%")

        QMessageBox.information(self, "Success", "Compression complete for all images.")

    def closeEvent(self, event):
        reply = QMessageBox.question(self, "Quit", "Are you sure you want to quit?", QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
        if reply == QMessageBox.Yes:
            event.accept()
        else:
            event.ignore()

if __name__ == "__main__":
    app = QApplication([])
    window = ImageCompressionApp()
    window.show()
    app.exec_()


Original Size: 5784384 bytes (5648.81 KB, 5.52 MB)
Compressed Size: 1230867 bytes (1202.02 KB, 1.17 MB)
Size Reduction: 4553517 bytes (78.72%)
